# 01: Extracción de Diputados

**Propósito:** Este *notebook* extrae la lista maestra de diputados para cada período legislativo, utilizando la API WSDL (`WSSala.asmx`) de la Cámara.

**Dependencias:**
* `data/01_raw/periodos_master.csv`: La lista maestra de períodos generada por el *notebook* `00_Extraction_Periods.ipynb`.

**Salidas (Artifacts):**
* Un archivo `diputados.csv` por cada período, guardado en `data/01_raw/[nombre_periodo]/diputados.csv`.

In [1]:
import pandas as pd
from pathlib import Path
from zeep import Client
import sys
import logging

# --- Configurar Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Importar lógica personalizada de /src ---
sys.path.append('../') 
try:
    from src.extraction_utils import get_diputados
    from src.common_utils import sanitize_filename
except ImportError:
    logging.error("Error: No se pudieron importar las funciones desde 'src/extraction_utils.py'.")
    logging.error("Asegúrese de mover 'estandarizar_df_diputados_x_periodo' y 'sanitize_filename' a ese archivo.")
    raise

In [2]:
# --- 1. Configuración de Rutas y Constantes ---

# URL del servicio web
WSDL_URL = "https://opendata.camara.cl/camaradiputados/WServices/WSDiputado.asmx?WSDL"

# Rutas del proyecto
ROOT = Path.cwd().parent
DATA_DIR_RAW = ROOT / "data" / "01_raw"

# --- Definir el ARCHIVO DE ENTRADA (Dependencia) ---
MASTER_PERIOD_FILE = DATA_DIR_RAW / "periodos_master.csv"

# Asegurarse que el directorio de datos exista
DATA_DIR_RAW.mkdir(parents=True, exist_ok=True)

logging.info(f"Ruta Raíz: {ROOT}")
logging.info(f"Directorio de Datos Raw: {DATA_DIR_RAW}")
logging.info(f"Archivo de períodos de entrada: {MASTER_PERIOD_FILE}")

2025-10-26 11:35:07,224 - INFO - Ruta Raíz: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-
2025-10-26 11:35:07,228 - INFO - Directorio de Datos Raw: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\01_raw
2025-10-26 11:35:07,229 - INFO - Archivo de períodos de entrada: C:\Users\angel\OneDrive\Documents\U\2025-2\Proyecto de Grado\Legislative-Voting-Behavior-Prediction-\data\01_raw\periodos_master.csv


## 2. Carga de Dependencias

Se inicializa el cliente de la API y se carga la lista maestra de períodos (`periodos_master.csv`). Si este archivo no existe, el *notebook* fallará, ya que es una dependencia crítica.

In [3]:
try:
    client = Client(WSDL_URL)
    logging.info(f"Cliente WSDL inicializado desde: {WSDL_URL}")
except Exception as e:
    logging.error(f"ERROR: No se pudo inicializar el cliente WSDL: {e}")
    raise

try:
    df_periodos = pd.read_csv(MASTER_PERIOD_FILE)
    logging.info(f"Se cargó la lista maestra de {len(df_periodos)} períodos.")
    display(df_periodos.head())
except FileNotFoundError:
    logging.error(f"ERROR FATAL: No se encontró el archivo de dependencia: {MASTER_PERIOD_FILE}")
    logging.error("Por favor, ejecute el notebook '00_Extraction_Periods.ipynb' primero.")
    raise

2025-10-26 11:35:07,711 - INFO - Cliente WSDL inicializado desde: https://opendata.camara.cl/camaradiputados/WServices/WSLegislativo.asmx?WSDL
2025-10-26 11:35:07,714 - INFO - Se cargó la lista maestra de 10 períodos.


,Id,Nombre,FechaInicio,FechaTermino
0,7,1965-1969,1965-03-11,1969-03-11 00:00:00
1,1,1990-1994,1990-03-11,1994-03-10 00:00:00
2,2,1994-1998,1994-03-11,1998-03-10 00:00:00
3,3,1998-2002,1998-03-11,2002-03-10 00:00:00
4,4,2002-2006,2002-03-11,2006-03-10 00:00:00


## 3. Bucle Principal de Extracción de Diputados

Iteramos sobre el `DataFrame` de períodos cargado. Para cada período, llamamos a la función de extracción (que vive en `/src`) y guardamos los resultados en la carpeta correspondiente.

In [4]:
logging.info(f"Iniciando extracción de diputados para {len(df_periodos)} períodos...")

for row in df_periodos.itertuples():
    pid = row.Id
    nombre_periodo = row.Nombre
    
    logging.info(f"Procesando Período: {pid} ({nombre_periodo})")
    
    try:
        # 1. Llamar a la función de lógica
        # Esta función (ahora en /src) debe contener la lógica de
        # 'get_diputados_periodo' y la estandarización/fusión.
        df_diputados = get_diputados(client, pid)

        # 2. Crear carpeta de salida
        nombre_carpeta = sanitize_filename(nombre_periodo)
        carpeta_salida = DATA_DIR_RAW / nombre_carpeta
        carpeta_salida.mkdir(parents=True, exist_ok=True)

        # 3. Definir y guardar archivo de salida
        ruta_salida = carpeta_salida / "diputados.csv"
        
        # Ordenar para consistencia
        df_diputados_sorted = df_diputados.sort_values(by=["Diputado.Id"])
        
        df_diputados_sorted.to_csv(ruta_salida, index=False, encoding="utf-8")
        
        logging.info(f"Guardado exitosamente: {ruta_salida} ({len(df_diputados)} diputados)")

    except Exception as e:
        logging.error(f"ERROR al procesar período {pid} ({nombre_periodo}): {e}")

logging.info("--- Extracción de diputados finalizada ---")

2025-10-26 11:35:07,731 - INFO - Iniciando extracción de diputados para 10 períodos...
2025-10-26 11:35:07,734 - INFO - Procesando Período: 7 (1965-1969)
2025-10-26 11:35:07,735 - ERROR - ERROR al procesar período 7 (1965-1969): Service has no operation 'retornarDiputadosXPeriodo'
2025-10-26 11:35:07,736 - INFO - Procesando Período: 1 (1990-1994)
2025-10-26 11:35:07,737 - ERROR - ERROR al procesar período 1 (1990-1994): Service has no operation 'retornarDiputadosXPeriodo'
2025-10-26 11:35:07,738 - INFO - Procesando Período: 2 (1994-1998)
2025-10-26 11:35:07,738 - ERROR - ERROR al procesar período 2 (1994-1998): Service has no operation 'retornarDiputadosXPeriodo'
2025-10-26 11:35:07,740 - INFO - Procesando Período: 3 (1998-2002)
2025-10-26 11:35:07,740 - ERROR - ERROR al procesar período 3 (1998-2002): Service has no operation 'retornarDiputadosXPeriodo'
2025-10-26 11:35:07,741 - INFO - Procesando Período: 4 (2002-2006)
2025-10-26 11:35:07,741 - ERROR - ERROR al procesar período 4 (200